In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings  # HuggingFace嵌入

load_dotenv()

# llm = ChatOpenAI(
#                     temperature=0,
#                     max_tokens=4096,
#                     presence_penalty=1.2,
#                     model='deepseek-chat',
#                     openai_api_key=os.getenv('DEEPSEEK_API_KEY'),
#                     openai_api_base="https://api.deepseek.com/v1",
#                 )

model = ChatOllama(
                    temperature=0,
                    num_predict=4096,
                    repeat_penalty=1.2,
                    model="llama3.2:3b"
                )

embedding = HuggingFaceEmbeddings(
            model_name="../../Embedding/maidalun/bce-embedding-base_v1",
            model_kwargs={"device": "cuda",
                          "trust_remote_code": True},
            encode_kwargs={"normalize_embeddings": True},
        )

c:\Users\farben\anaconda3\envs\lclg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define model and tools for the graph

In [5]:
from typing import Literal

from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

tools = [get_weather]

# MongoDB checkpointer usage

## With a connection string

In [6]:
from langgraph.checkpoint.mongodb import MongoDBSaver

MONGODB_URI = "localhost:27017"  # replace this with your connection string

with MongoDBSaver.from_conn_string(MONGODB_URI) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    response = graph.invoke(
        {"messages": [("human", "what's the weather in sf")]}, config
    )

In [7]:
response

{'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='6de18e94-c95a-4a72-8164-1620fe68afc1'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T05:54:10.4119508Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1695946400, 'load_duration': 1470938100, 'prompt_eval_count': 158, 'prompt_eval_duration': 43269000, 'eval_count': 17, 'eval_duration': 180761000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_weather', arguments={'city': 'sf'}))])}, id='run-91bbe58a-128c-4156-9ba7-f0e92558cf28-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '5052ebca-bdb1-4108-8214-253a92d35b06', 'type': 'tool_call'}], usage_metadata={'input_tokens': 158, 'output_tokens': 17, 'total_tokens': 175}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='85cd8d5b-d2ac-4083-88c2-4d14

## Using the MongoDB client

In [8]:
from pymongo import MongoClient

mongodb_client = MongoClient(MONGODB_URI)

checkpointer = MongoDBSaver(mongodb_client)
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
response = graph.invoke({"messages": [("user", "What's the weather in sf?")]}, config)

In [10]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='522d5d15-102e-4e6d-8f77-4e5fab1f19a2'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T06:06:14.9819419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1722413400, 'load_duration': 1497880500, 'prompt_eval_count': 159, 'prompt_eval_duration': 42521000, 'eval_count': 17, 'eval_duration': 180690000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_weather', arguments={'city': 'sf'}))])}, id='run-53af0279-be01-4e65-828a-6402326f5422-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '960b5aa5-fb29-4ccd-a305-2d695912a983', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 17, 'total_tokens': 176}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='c4edc31a-55d2-47e3-8d6f-238

In [11]:
# Retrieve the latest checkpoint for the given thread ID
# To retrieve a specific checkpoint, pass the checkpoint_id in the config
checkpointer.get_tuple(config)

CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efbd063-09b7-60e3-8003-4a920e5ed3c4'}}, checkpoint={'v': 1, 'ts': '2024-12-18T06:06:15.472150+00:00', 'id': '1efbd063-09b7-60e3-8003-4a920e5ed3c4', 'channel_values': {'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='522d5d15-102e-4e6d-8f77-4e5fab1f19a2'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T06:06:14.9819419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1722413400, 'load_duration': 1497880500, 'prompt_eval_count': 159, 'prompt_eval_duration': 42521000, 'eval_count': 17, 'eval_duration': 180690000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': [{'function': {'name': 'get_weather', 'arguments': {'city': 'sf'}}}]}}, id='run-53af0279-be01-4e65-828a-6402326f5422-0', tool_calls=[{'name': 'get_weather', 'args': {'cit

In [12]:
# Remember to close the connection after you're done
mongodb_client.close()

## Using an async connection

In [16]:
from langgraph.checkpoint.mongodb.aio import AsyncMongoDBSaver

async with AsyncMongoDBSaver.from_conn_string(MONGODB_URI) as checkpointer:
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "3"}}
    response = await graph.ainvoke(
        {"messages": [("user", "What's the weather in sf?")]}, config
    )

In [17]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='1d240e9b-7b02-47d7-92cd-923d0c4be140'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T06:09:19.1831026Z', 'done': True, 'done_reason': 'stop', 'total_duration': 464944500, 'load_duration': 17603700, 'prompt_eval_count': 159, 'prompt_eval_duration': 268017000, 'eval_count': 17, 'eval_duration': 178482000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': [{'function': {'name': 'get_weather', 'arguments': {'city': 'sf'}}}]}}, id='run-01893763-6380-4e0c-8a1d-ffc80b260fcc-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'fd1a883e-b909-40f9-a7a9-07d4cdb4f21d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 17, 'total_tokens': 176}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='eed24b7b-3a93-4ae6-854e-70bbebb

## Using the async MongoDB client

In [18]:
from pymongo import AsyncMongoClient

async_mongodb_client = AsyncMongoClient(MONGODB_URI)

checkpointer = AsyncMongoDBSaver(async_mongodb_client)
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "4"}}
response = await graph.ainvoke(
    {"messages": [("user", "What's the weather in sf?")]}, config
)

In [19]:
response

{'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='8d624372-ac37-4c34-9675-d7e85ca1cd15'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T06:09:49.7515791Z', 'done': True, 'done_reason': 'stop', 'total_duration': 574915100, 'load_duration': 29585300, 'prompt_eval_count': 159, 'prompt_eval_duration': 364537000, 'eval_count': 17, 'eval_duration': 179470000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_weather', arguments={'city': 'sf'}))])}, id='run-4cdd1b30-3930-4f0e-8a8d-330cd662be86-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '94304992-5780-47a1-a8a1-6a25447c867e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 17, 'total_tokens': 176}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='9b7b741d-4258-46f0-b743-311b4

In [20]:
# Retrieve the latest checkpoint for the given thread ID
# To retrieve a specific checkpoint, pass the checkpoint_id in the config
latest_checkpoint = await checkpointer.aget_tuple(config)
print(latest_checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efbd06b-09ff-6f73-8003-d9ebe3919c37'}}, checkpoint={'v': 1, 'ts': '2024-12-18T06:09:50.250379+00:00', 'id': '1efbd06b-09ff-6f73-8003-d9ebe3919c37', 'channel_values': {'messages': [HumanMessage(content="What's the weather in sf?", additional_kwargs={}, response_metadata={}, id='8d624372-ac37-4c34-9675-d7e85ca1cd15'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-18T06:09:49.7515791Z', 'done': True, 'done_reason': 'stop', 'total_duration': 574915100, 'load_duration': 29585300, 'prompt_eval_count': 159, 'prompt_eval_duration': 364537000, 'eval_count': 17, 'eval_duration': 179470000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': [{'function': {'name': 'get_weather', 'arguments': {'city': 'sf'}}}]}}, id='run-4cdd1b30-3930-4f0e-8a8d-330cd662be86-0', tool_calls=[{'name': 'get_weather', 'args': {'city'

In [21]:
# Remember to close the connection after you're done
await async_mongodb_client.close()